**Определение уровня лояльности клиента телекоммуникационной компании.**

**Описание проекта**

Заказчик исследования - телекоммуникационная компания, работающая в СНГ. В данном исследовании нужно определить уровень потребительской лояльности клиентов из России.

**Цель проекта:**

Определить уровень потребительской лояльности среди клиентов из России.

**Ход исследования:**

• Шаг 1: Подключение к базе данных SQLite с файлом .db. Так же в таблицу требуется добавить и рассчитать столбцы nps_group и is_new. 

• Шаг 2: выполните предобработку данных, проанализировать таблицу на дубликаты, откорректировать типы данных в столбцах.

• Шаг 3: выгрузка данных из таблицы для создания дашборда.

• Шаг 4: Создание дашборда в Tableau для представления информации о NPS и его изменении в зависимости от пользовательских признаков.

• Шаг 5: Ответы на вопросы с помощью дашборда, оформление в виде презентации. Публикация дашборда на сайте Tableau Public для проверки доступности.

Я проверю данные на ошибки и оценю их влияние на исследование при подготовке к анализу в Tableau. Далее при помощи визуализации я отвечаю на вопросы проекта:
- Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
- Какой общий NPS среди всех опрошенных?
- Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?
Конечные результаты оформляю в презентацию и делаю выводы о проведенной работе.

 

## check

**Шаг 1. Загрузите данные и изучите общую информацию**

На данном этапе я выгрузила данные при помощи SQL в одну таблицу. Так же я добавила в нее еще два столбца которые предварительно рассчитала. Столбец is_new хранит информацию является ли клиент новым, а новым клиент считается, если количество дней «жизни» составляет не более 365 дней. И столбец  nps_group, который поделит значения на следующие группы: 
9-10 баллов — «cторонники» (англ. promoters);
7-8 баллов — «нейтралы» (англ. passives);
0-6 баллов — «критики» (англ. detractors). 

Так же на данном этапе я добавляю столбец с полом разделяя всех клиентов женский, мужской и неопределенный пол.

In [2]:
import os
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

# путь к БД на вашем компьютере (например, в той же папке, что и тетрадь)
path_to_db_local = 'telecomm_csi.db'
# путь к БД на платформе
path_to_db_platform = '/datasets/telecomm_csi.db'
# итоговый путь к БД
path_to_db = None

# если путь на вашем компьютере ведёт к БД, то он становится итоговым
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
# иначе: если путь на платформе ведёт к БД, то он становится итоговым
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
# иначе выводится сообщение о том, что файл не найден
else:
    raise Exception('Файл с базой данных SQLite не найден!')

# если итоговый путь не пустой
if path_to_db:
    # то создаём подключение к базе
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)
    
    # пример запроса
    query = """
    SELECT u.*,
       age_segment.title AS age_title,
       traffic_segment.title AS traffic_title,
       lifetime_segment.title AS lifetime_title,
       l.country,
       l.city,
       CASE
           WHEN lt_day <= 365 THEN 'новый'
           ELSE 'старый'
       END AS is_new,
       CASE
           WHEN nps_score BETWEEN 9 AND 10 THEN 'сторонники'
           WHEN nps_score BETWEEN 7 AND 8 THEN 'нейтралы'
           WHEN nps_score BETWEEN 0 AND 6 THEN 'критики'
       END AS nps_group,
       CASE 
           WHEN gender_segment = 1.0 THEN 'женский'
           WHEN gender_segment = 0.0 THEN 'мужской'
           WHEN gender_segment IS NULL THEN 'unknown'
           ELSE 'unknown' 
       END AS gender
       FROM user u
LEFT JOIN age_segment ON u.age_gr_id = age_segment.age_gr_id
LEFT JOIN traffic_segment ON u.tr_gr_id = traffic_segment.tr_gr_id
LEFT JOIN lifetime_segment ON u.lt_gr_id = lifetime_segment.lt_gr_id
INNER JOIN location l ON u.location_id = l.location_id;
"""

    # создаём датафрейм по данным запроса
    data = pd.read_sql(query, engine) 

In [3]:
display(data)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score,age_title,traffic_title,lifetime_title,country,city,is_new,nps_group,gender
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,10,05 45-54,04 1-5,08 36+,Россия,Уфа,старый,сторонники,женский
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,10,05 45-54,04 1-5,08 36+,Россия,Киров,старый,сторонники,мужской
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,10,06 55-64,08 20-25,06 13-24,Россия,Москва,старый,сторонники,мужской
3,A004TB,4190,44.0,1.0,IOS,SMARTPHONE,38,4,4,8,10,04 35-44,03 0.1-1,08 36+,Россия,РостовнаДону,старый,сторонники,женский
4,A004XT,1163,24.0,0.0,ANDROID,SMARTPHONE,39,2,6,8,10,02 16-24,05 5-10,08 36+,Россия,Рязань,старый,сторонники,мужской
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502488,ZZZKLD,1249,54.0,1.0,ANDROID,SMARTPHONE,28,5,5,8,5,05 45-54,04 1-5,08 36+,Россия,Москва,старый,критики,женский
502489,ZZZLWY,129,31.0,0.0,ANDROID,SMARTPHONE,28,3,5,4,8,03 25-34,04 1-5,04 4-6,Россия,Москва,новый,нейтралы,мужской
502490,ZZZQ5F,522,36.0,0.0,ANDROID,SMARTPHONE,47,4,10,6,10,04 35-44,09 25-30,06 13-24,Россия,Сургут,старый,сторонники,мужской
502491,ZZZQ8E,2936,37.0,1.0,ANDROID,SMARTPHONE,53,4,18,8,9,04 35-44,17 65-70,08 36+,Россия,УланУдэ,старый,сторонники,женский


Далее я рассмотрю данные более детально и проведу предообработку и проверю на дубликаты.

In [4]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         502493 non-null  object 
 1   lt_day          502493 non-null  int64  
 2   age             501939 non-null  float64
 3   gender_segment  501192 non-null  float64
 4   os_name         502493 non-null  object 
 5   cpe_type_name   502493 non-null  object 
 6   location_id     502493 non-null  int64  
 7   age_gr_id       502493 non-null  int64  
 8   tr_gr_id        502493 non-null  int64  
 9   lt_gr_id        502493 non-null  int64  
 10  nps_score       502493 non-null  int64  
 11  age_title       502493 non-null  object 
 12  traffic_title   502493 non-null  object 
 13  lifetime_title  502493 non-null  object 
 14  country         502493 non-null  object 
 15  city            502493 non-null  object 
 16  is_new          502493 non-null  object 
 17  nps_group 

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score,age_title,traffic_title,lifetime_title,country,city,is_new,nps_group,gender
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,10,05 45-54,04 1-5,08 36+,Россия,Уфа,старый,сторонники,женский
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,10,05 45-54,04 1-5,08 36+,Россия,Киров,старый,сторонники,мужской
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,10,06 55-64,08 20-25,06 13-24,Россия,Москва,старый,сторонники,мужской
3,A004TB,4190,44.0,1.0,IOS,SMARTPHONE,38,4,4,8,10,04 35-44,03 0.1-1,08 36+,Россия,РостовнаДону,старый,сторонники,женский
4,A004XT,1163,24.0,0.0,ANDROID,SMARTPHONE,39,2,6,8,10,02 16-24,05 5-10,08 36+,Россия,Рязань,старый,сторонники,мужской


На данном этапе можно заметить, что колонка с названиями городов выглядит не совсем корректно, так же я хочу заменить тип данных в столбце 'agee'.

## check

**Шаг 2. Выполните предобработку данных**

In [5]:
data.isnull().sum()

user_id              0
lt_day               0
age                554
gender_segment    1301
os_name              0
cpe_type_name        0
location_id          0
age_gr_id            0
tr_gr_id             0
lt_gr_id             0
nps_score            0
age_title            0
traffic_title        0
lifetime_title       0
country              0
city                 0
is_new               0
nps_group            0
gender               0
dtype: int64

Основные пропуски в столбцах gender_segment и age, возможно клиент просто пропустил эту строку при заполнении данных. 

In [6]:
data = data.drop(['location_id', 'age_gr_id', 'tr_gr_id', 'lt_gr_id'], axis=1)

На данном этапе я удаляю основные ключи, по которым ранее я объединяла таблицы.

Проверка на дубликаты

In [7]:
data.duplicated().sum()

0

Явные дубликаты отсутствуют, стоит проверить таблицу на неявные дубли.

In [8]:
data['country'].unique()

array(['Россия'], dtype=object)

In [9]:
data['city'].unique()

array(['Уфа', 'Киров', 'Москва', 'РостовнаДону', 'Рязань', 'Омск',
       'СанктПетербург', 'Волгоград', 'Тольятти', 'Казань', 'Самара',
       'Красноярск', 'Екатеринбург', 'Калуга', 'Краснодар', 'Иркутск',
       'Пермь', 'Владимир', 'Ижевск', 'Тюмень', 'Оренбург',
       'НижнийНовгород', 'Брянск', 'Челябинск', 'Астрахань', 'Сургут',
       'Тверь', 'Новосибирск', 'НабережныеЧелны', 'Махачкала', 'Воронеж',
       'Курск', 'Владивосток', 'Балашиха', 'Пенза', 'Калининград', 'Тула',
       'Саратов', 'Кемерово', 'Белгород', 'Барнаул', 'Чебоксары',
       'Архангельск', 'Томск', 'Ярославль', 'Ульяновск', 'Хабаровск',
       'Грозный', 'Ставрополь', 'Липецк', 'Новокузнецк', 'Якутск',
       'УланУдэ', 'Сочи', 'Иваново', 'НижнийТагил', 'Смоленск',
       'Волжский', 'Магнитогорск', 'Чита', 'Череповец', 'Саранск'],
      dtype=object)

In [10]:
replacement_dict = {
    'РостовнаДону': 'Ростов-на-Дону',
    'СанктПетербург': 'Санкт-Петербург',
    'УланУдэ': 'Улан-Удэ',
    'НижнийНовгород': 'Нижний Новгород',
    'НабережныеЧелны': 'Набережные Челны',
    'НижнийТагил': 'Нижний Тагил'
}

data['city'] = data['city'].replace(replacement_dict, regex=True)

In [12]:
data['city'].unique()

array(['Уфа', 'Киров', 'Москва', 'Ростов-на-Дону', 'Рязань', 'Омск',
       'Санкт-Петербург', 'Волгоград', 'Тольятти', 'Казань', 'Самара',
       'Красноярск', 'Екатеринбург', 'Калуга', 'Краснодар', 'Иркутск',
       'Пермь', 'Владимир', 'Ижевск', 'Тюмень', 'Оренбург',
       'Нижний Новгород', 'Брянск', 'Челябинск', 'Астрахань', 'Сургут',
       'Тверь', 'Новосибирск', 'Набережные Челны', 'Махачкала', 'Воронеж',
       'Курск', 'Владивосток', 'Балашиха', 'Пенза', 'Калининград', 'Тула',
       'Саратов', 'Кемерово', 'Белгород', 'Барнаул', 'Чебоксары',
       'Архангельск', 'Томск', 'Ярославль', 'Ульяновск', 'Хабаровск',
       'Грозный', 'Ставрополь', 'Липецк', 'Новокузнецк', 'Якутск',
       'Улан-Удэ', 'Сочи', 'Иваново', 'Нижний Тагил', 'Смоленск',
       'Волжский', 'Магнитогорск', 'Чита', 'Череповец', 'Саранск'],
      dtype=object)

На данном этапе я привела названия городов к читаемому и правильному виду.

In [13]:
data['age'].unique()

array([45., 53., 57., 44., 24., 42., 35., 36., 54., 39., 21., 27., 60.,
       34., 47., 37., 43., 33., 31., 25., 51., 28., 41., 40., 46., 48.,
       32., 30., 52., 59., 26., 50., 62., 29., 55., 22., 38., 56., 23.,
       49., 66., 74., 75., 17., 65., 64., 69., 58., 20., 19., 80., 70.,
       81., 63., 67., 68., 72., 15., 79., 18., 73., nan, 14., 71., 61.,
       16., 77., 13., 76., 10., 78., 12., 82., 11., 83., 89., 84., 85.,
       87., 86.])

In [14]:
data['age'] = data['age'].fillna(-1)

data = data.replace([np.inf, -np.inf], np.nan).dropna(subset=['age'])

data['age'] = data['age'].astype(int)

In [15]:
display(data)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,nps_score,age_title,traffic_title,lifetime_title,country,city,is_new,nps_group,gender
0,A001A2,2320,45,1.0,ANDROID,SMARTPHONE,10,05 45-54,04 1-5,08 36+,Россия,Уфа,старый,сторонники,женский
1,A001WF,2344,53,0.0,ANDROID,SMARTPHONE,10,05 45-54,04 1-5,08 36+,Россия,Киров,старый,сторонники,мужской
2,A003Q7,467,57,0.0,ANDROID,SMARTPHONE,10,06 55-64,08 20-25,06 13-24,Россия,Москва,старый,сторонники,мужской
3,A004TB,4190,44,1.0,IOS,SMARTPHONE,10,04 35-44,03 0.1-1,08 36+,Россия,Ростов-на-Дону,старый,сторонники,женский
4,A004XT,1163,24,0.0,ANDROID,SMARTPHONE,10,02 16-24,05 5-10,08 36+,Россия,Рязань,старый,сторонники,мужской
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502488,ZZZKLD,1249,54,1.0,ANDROID,SMARTPHONE,5,05 45-54,04 1-5,08 36+,Россия,Москва,старый,критики,женский
502489,ZZZLWY,129,31,0.0,ANDROID,SMARTPHONE,8,03 25-34,04 1-5,04 4-6,Россия,Москва,новый,нейтралы,мужской
502490,ZZZQ5F,522,36,0.0,ANDROID,SMARTPHONE,10,04 35-44,09 25-30,06 13-24,Россия,Сургут,старый,сторонники,мужской
502491,ZZZQ8E,2936,37,1.0,ANDROID,SMARTPHONE,9,04 35-44,17 65-70,08 36+,Россия,Улан-Удэ,старый,сторонники,женский


In [16]:
desktop_path = 'C:\\nps.csv'

try:
    data.to_csv(desktop_path, index=False)
    print("Файл успешно сохранен.")
except Exception as e:
    print(f"Произошла ошибка при сохранении файла: {e}")

Файл успешно сохранен.


## check

Ссылка на проек

https://public.tableau.com/shared/C39QKJQF4?:display_count=n&:origin=viz_share_link